In [36]:
import json
import requests
import time

## Collect all unigrams

In [5]:
unigrams=[]
for i in range(6):
    with open("/home/cline/NYT_SPEED/Unigrams/{0}.json".format(i)) as f:
        unis = json.load(f)
    unigrams.extend(unis)

In [6]:
len(unigrams)

53581

There are 53581 articles worth of unigrams.

## Get the Solr Metadata

In [9]:
def get_solr_data(uni_id):
    
    ## uni id is in [0, 1, ..., 53580]

    aid = unigrams[uni_id]['aid']
    query = "http://localhost:8983/solr/derived/select?q=aid%3A{0}&wt=json&indent=true".format(aid)
    r = requests.get(query)
    
    return(r)

In [10]:
r = get_solr_data(0)

## Get the NYT Metadata

But first, you have to go get an API key. There is no wait or cost.

In [12]:
with open("NYT.key") as key:
    for line in key:
        NYT_KEY = line

In [20]:
def get_NYT_data(r, NYT_KEY):
    ttl = r.json()['response']['docs'][0]['title']
    date = r.json()['response']['docs'][0]['publication_date'][:10]
    nytq = 'http://api.nytimes.com/svc/search/v2/articlesearch.json?\
fq=headline:("{0}") AND pub_date:{1}&api-key={2}'.format(ttl,date, NYT_KEY)
    nyt = requests.get(nytq)
    return(nyt)

In [51]:
nyt_meta = get_NYT_data(r, NYT_KEY)
nyt_meta.json()

In [23]:
def get_all_metadata(uni_id, NYT_KEY):
    solr_meta = get_solr_data(uni_id)
    nyt_meta = get_NYT_data(solr_meta, NYT_KEY)
    return(solr_meta, nyt_meta)
    

In [50]:
for i in range(20):
    s, n = get_all_metadata(i, NYT_KEY)
    try:
        print(n.json()['response']['docs'][0]["news_desk"], n.json()['response']['docs'][0]["section_name"])
    except IndexError:
        print(i) # print the uni_index if there's an issue
    time.sleep(1)

None None
None None
None None
None None
None None
None None
None None
None None
None None
None None
None None
None None
None None
None None
None None
None None
None None
None None
None None
None None


It looks like these articles don't have the section name or news desk metadata. They may be too old?